In [ ]:
from basicsr.trainer import Surgeon, SurgeonTrainer
from basicsr.data import restore_volume, semi_synthetic_creation, \
     generate_raw_data, generate_zs_dataset, adjust_contrast, denoised_semi_synthetic_creation
from basicsr.model_train import trainer_train,restore
from easydict import EasyDict
import os
import tifffile
from oct.utils import generate_dataset, stack
import cv2
from basicsr.utils.dataset import normalize, resize
import numpy as np


c:\Users\rkulothu\Documents\Repos\SSAI-oct\SSAI-3D\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
SSAI_PATH = "C:/Users/rkulothu/Documents/Repos/SSAI-oct/SSAI-3D"
# SSAI_PATH = "/Users/ramachandrank/Repos/SSAI-3D/"
raw_tif_path = f"{SSAI_PATH}/raw_data/0704_OD_B-scans_log_16bits.tif"
project_depth = 4
data_save_path = "data_e2e"
results_save_path = "results"

In [3]:
generate_dataset.create_projected_data_with_1step_slide(raw_tif_path, data_save_path, project_depth = 4, projected_dir = "projected_data")

In [ ]:
angles = [5, 10, -5, -10]

In [ ]:
for angle in angles:
    angle_data_save_path = f"angle_{angle}" + data_save_path
    angle_results_save_path = f"angle_{angle}" + results_save_path
    
    
    generate_dataset.create_synthetic_data_from_slices(f"{data_save_path}/projected_data", angle_data_save_path, 2, 1, 2, True, False, angle)
    generate_dataset.create_downsampled_data_from_slices(f"{data_save_path}/projected_data", angle_data_save_path, 1, 2)
    
    
    generate_dataset.create_zs_dataset(angle_data_save_path)
    configs = EasyDict({
    'model_path' : f'{SSAI_PATH}/pretrained_models/NAFNet-REDS-width64.pth',
    'lq_pth': os.path.join(angle_data_save_path, 'zs_lq'),
    'gt_pth': os.path.join(angle_data_save_path, 'zs_gt'),
    'lr': 1e-4
    })
    operation = Surgeon(configs)
    operation.get_zeroshot_information()
    surgeon_trainer = SurgeonTrainer()
    elementwise_input = operation.input_dict.copy()
    rank = surgeon_trainer.forward_all(elementwise_input)


    trainer_train(rank, angle_data_save_path)


    generate_dataset.generate_oct_raw_data(
        raw_pth = raw_tif_path, save_pth= angle_data_save_path, dr = 2,
        xy_required=True, xz_required=True, yz_required=True
        )
    in_pth_xz = os.path.join(angle_data_save_path, 'test_xz')
    out_pth_xz = os.path.join(angle_results_save_path, 'test_xz')
    restore(input_pth = in_pth_xz,\
        output_pth = out_pth_xz,\
        model_pth =  'C:/Users/rkulothu/Documents/Repos/SSAI-oct/SSAI-3D/pretrained_models/NAFNet-REDS-width64.pth')
    out_pth_xz = os.path.join(angle_results_save_path, 'test_xz')
    stack.create_3d_tiff(out_pth_xz, os.path.join(angle_results_save_path ,f"angle_{angle}_rec_xz.tiff"))


    generate_dataset.generate_oct_raw_data(raw_pth = os.path.join(angle_results_save_path ,f"angle_{angle}_rec_xz.tiff"), 
                                           save_pth= os.path.join(angle_data_save_path, "restored_input"), dr = 2, yz_required=True)
    in_pth_xy = os.path.join(angle_data_save_path, "restored_input", 'test_yz')
    out_pth_xy = os.path.join(angle_results_save_path, "restored_input", 'test_xy')
    restore(input_pth = in_pth_xy,\
        output_pth = out_pth_xy,\
        model_pth =  'C:/Users/rkulothu/Documents/Repos/SSAI-oct/SSAI-3D/pretrained_models/NAFNet-REDS-width64.pth')
    stack.create_3d_tiff(out_pth_xy, os.path.join(angle_results_save_path ,f"angle_{angle}_16bit_log_final.tiff"))
